In [1]:
import numpy as np
import pandas as pd
from pathlib import Path  

In [198]:
#df2 = pd.ExcelFile('../../data/raw/Jones-Kammen-2014-Zip-City-County-Results.xlsx')
cfc_df = pd.read_excel('../../data/raw/Jones-Kammen-2014-Zip-City-County-Results.xlsx', sheet_name= 'County results')
cfc_df.head()

,Population,State,County,electricity (kWh),Nat. Gas (cu.ft.),FUELOIL (gallons),Vehicle miles traveled,Transport (tCO2e/yr),Housing (tCO2e/yr),Food (tCO2e/yr),Goods (tCO2e/yr),Services (tCO2e/yr),Total Household Carbon Footprint (tCO2e/yr),Households,Total County Carbon Footprint (tCO2e/yr)
0,10,AK,JUNEAU,4128.496254,26033.750948,72.354330,8074.000000,5.925587,6.065464,3.750000,3.054273,3.466416,22.261739,85.0,1892.247853
1,4336,AK,YUKON KOYUKUK,5581.508536,26607.262942,345.595186,10363.603552,7.543985,5.364597,8.801451,4.704034,4.957767,31.371833,1132.0,35512.914997
2,15981,AK,BETHEL,5938.450363,26562.320038,556.653438,11799.094049,8.452924,3.028379,11.553512,5.435957,5.560463,34.031235,1239.0,42164.700168
3,1745,AK,LAKE AND PENINSULA,6091.910223,25538.669840,579.568249,11820.514040,8.646400,8.184707,9.401811,5.201408,5.533073,36.967400,819.0,30276.300664
4,5098,AK,ALEUTIANS WEST,5361.596806,23406.596842,502.171462,17587.886230,13.408990,2.364099,7.677670,6.683501,7.774904,37.909165,352.0,13344.025940


In [5]:
#create new columns for data
cfc_df.insert(loc = 0, column = 'Transport Total (tCO2e/yr)', value = cfc_df['Transport (tCO2e/yr)']*cfc_df['Households'])
cfc_df.insert(loc = 0, column = 'Housing Total (tCO2e/yr)', value = cfc_df['Housing (tCO2e/yr)']*cfc_df['Households'])
cfc_df.insert(loc = 0, column = 'Food Total (tCO2e/yr)', value = cfc_df['Food (tCO2e/yr)']*cfc_df['Households'])
cfc_df.insert(loc = 0, column = 'Goods Total (tCO2e/yr)', value = cfc_df['Goods (tCO2e/yr)']*cfc_df['Households'])
cfc_df.insert(loc = 0, column = 'Services Total (tCO2e/yr)', value = cfc_df['Services (tCO2e/yr)']*cfc_df['Households'])

In [6]:
## change misnomer 'Total Household Carbon Footprint (tCO2e/yr)' to 'Total Per-Household Carbon Footprint (tCO2e/yr)'
cfc_df.rename(columns = {'Total Household Carbon Footprint (tCO2e/yr)' : 'Total Per-Household Carbon Footprint (tCO2e/yr)'}, inplace = True)

In [148]:
cfc_df_totals = cfc_df[['Population', 'State', 'County', 'electricity (kWh)', 'Nat. Gas (cu.ft.)', 'FUELOIL (gallons)', 'Vehicle miles traveled', 
                 'Transport Total (tCO2e/yr)','Housing Total (tCO2e/yr)', 'Food Total (tCO2e/yr)','Goods Total (tCO2e/yr)', 
                 'Services Total (tCO2e/yr)', # 'New Total Carbon Footprints per MSA', 
                 'Total Per-Household Carbon Footprint (tCO2e/yr)', 'Households', 'Total County Carbon Footprint (tCO2e/yr)']]

In [8]:
cfc_df_totals.head()

,Population,State,County,electricity (kWh),Nat. Gas (cu.ft.),FUELOIL (gallons),Vehicle miles traveled,Transport Total (tCO2e/yr),Housing Total (tCO2e/yr),Food Total (tCO2e/yr),Goods Total (tCO2e/yr),Services Total (tCO2e/yr),Total Per-Household Carbon Footprint (tCO2e/yr),Households,Total County Carbon Footprint (tCO2e/yr)
0,10,AK,JUNEAU,4128.496254,26033.750948,72.354330,8074.000000,503.674865,515.564451,318.750000,259.613201,294.645335,22.261739,85.0,1892.247853
1,4336,AK,YUKON KOYUKUK,5581.508536,26607.262942,345.595186,10363.603552,8539.790566,6072.723929,9963.242131,5324.966486,5612.191884,31.371833,1132.0,35512.914997
2,15981,AK,BETHEL,5938.450363,26562.320038,556.653438,11799.094049,10473.173168,3752.161045,14314.801735,6735.150436,6889.413784,34.031235,1239.0,42164.700168
3,1745,AK,LAKE AND PENINSULA,6091.910223,25538.669840,579.568249,11820.514040,7081.401965,6703.275336,7700.083117,4259.953401,4531.586844,36.967400,819.0,30276.300664
4,5098,AK,ALEUTIANS WEST,5361.596806,23406.596842,502.171462,17587.886230,4719.964562,832.162797,2702.539725,2352.592510,2736.766345,37.909165,352.0,13344.025940


In [79]:
## msa--municipality (county) mapping
df = pd.read_csv('../../data/tidy/msa-municipality.csv', delimiter=',', skiprows=0, low_memory=False)
df.head()

,msa,municipality
0,"Abilene, TX","['Callahan County, TX', 'Jones County, TX', 'T..."
1,"Akron, OH","['Portage County, OH', 'Summit County, OH']"
2,"Albany, GA","['Dougherty County, GA', 'Lee County, GA', 'Te..."
3,"Albany-Lebanon, OR","['Linn County, OR']"
4,"Albany-Schenectady-Troy, NY","['Albany County, NY', 'Rensselaer County, NY',..."


In [80]:
# use the explode function to split the municipality column into separate rows
df["municipality"] = df["municipality"].apply(eval)
msa_muni_exploded = df.explode('municipality')
msa_muni_exploded = msa_muni_exploded.reset_index(drop=True) # reset the index of the dataframe
msa_muni_exploded.head()

,msa,municipality
0,"Abilene, TX","Callahan County, TX"
1,"Abilene, TX","Jones County, TX"
2,"Abilene, TX","Taylor County, TX"
3,"Akron, OH","Portage County, OH"
4,"Akron, OH","Summit County, OH"


In [83]:
## Get column of states only
msa_muni_exploded['state'] = msa_muni_exploded['municipality'].str.replace('County', '')
msa_muni_exploded['state'] = msa_muni_exploded['state'].str.split(',', expand=True).iloc[:,1]
msa_muni_exploded['state'] = msa_muni_exploded['state'].str.strip()

In [84]:
msa_muni_exploded['state'].unique()

array(['TX', 'OH', 'GA', 'OR', 'NY', 'NM', 'LA', 'NJ', 'PA', 'IA', 'AK',
       'MI', 'AL', 'WI', 'NC', 'SC', 'CA', 'MD', 'ME', 'MA', 'WV', 'WA',
       'MT', 'ND', 'VA', 'IL', 'IN', 'ID', 'NH', 'CO', 'KY', 'CT', 'VT',
       'FL', 'MO', 'NV', 'WY', 'TN', 'DE', 'MN', 'OK', 'AR', 'AZ', 'NE',
       'MS', 'HI', 'KS', 'UT', 'RI', 'SD', '', 'DC'], dtype=object)

In [85]:
## Get county column and format county names to look like those in cfc_df
msa_muni_exploded['municipality'] = msa_muni_exploded['municipality'].str.replace('County', '')
msa_muni_exploded['municipality'] = msa_muni_exploded['municipality'].str.split(',', expand=True).iloc[:,0]
msa_muni_exploded['municipality'] = msa_muni_exploded['municipality'].str.upper()

In [34]:
## CHECK: Compare both lists to see if they are the same (use setdiff, etc)
# msa_muni_exploded['municipality'].unique()
# cfc_df['County'].unique()

In [70]:
len(msa_muni_exploded)

1181

In [150]:
cfc_df_totals.loc[:,'MSA'] = np.nan

/tmp/ipykernel_195905/3097761227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cfc_df_totals.loc[:,'MSA'] = np.nan


In [151]:
for i in cfc_df_totals.index:
    #print(i)
    msa_assign = msa_muni_exploded.loc[(msa_muni_exploded['municipality'] == cfc_df_totals.loc[i,'County'])  &
                                       (msa_muni_exploded['state'] == cfc_df_totals.loc[i,'State']), 'msa']
    
    if msa_assign.empty == False:
        # print(msa_assign.iloc[0])
        cfc_df_totals.at[i,'MSA'] = msa_assign.iloc[0]

In [182]:
## aggregate by MSA
cfc_msa = cfc_df_totals.groupby(by=["MSA"]).sum()
# cfc_msa.head()

/tmp/ipykernel_195905/1102604514.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  cfc_msa = cfc_df_totals.groupby(by=["MSA"]).sum()


In [183]:
cfc_msa = cfc_msa.drop(['Total Per-Household Carbon Footprint (tCO2e/yr)','Total County Carbon Footprint (tCO2e/yr)'] ,axis=1)

In [184]:
#create new columns for data (We may want to consider keeping the column names formal for now (similar to the original))
cfc_msa.insert(loc = 11, column = 'cf_transport_phh', value = cfc_msa['Transport Total (tCO2e/yr)']/cfc_msa['Households'])
cfc_msa.insert(loc = 12, column = 'cf_housing_phh', value = cfc_msa['Housing Total (tCO2e/yr)']/cfc_msa['Households'])
cfc_msa.insert(loc = 13, column = 'cf_food_phh', value = cfc_msa['Food Total (tCO2e/yr)']/cfc_msa['Households'])
cfc_msa.insert(loc = 14, column = 'cf_goods_phh', value = cfc_msa['Goods Total (tCO2e/yr)']/cfc_msa['Households'])
cfc_msa.insert(loc = 15, column = 'cf_services_phh', value = cfc_msa['Services Total (tCO2e/yr)']/cfc_msa['Households'])

In [185]:
cfc_msa.insert(loc = 16, column = 'cf', value = (cfc_msa['Transport Total (tCO2e/yr)'] + 
                                                 cfc_msa['Transport Total (tCO2e/yr)'] + 
                                                 cfc_msa['Food Total (tCO2e/yr)'] + 
                                                 cfc_msa['Goods Total (tCO2e/yr)'] + 
                                                 cfc_msa['Services Total (tCO2e/yr)'])) ## total MSA carbon footprint               

In [186]:
cfc_msa.insert(loc = 16, column = 'cf_phh', value = (cfc_msa['cf']/cfc_msa['Households']))

In [187]:
cfc_msa = cfc_msa.drop(['Transport Total (tCO2e/yr)', 'Housing Total (tCO2e/yr)', 'Food Total (tCO2e/yr)', 'Goods Total (tCO2e/yr)', 'Services Total (tCO2e/yr)'] ,axis=1)

In [195]:
#cfc_msa.rename(columns = {'Total Household Carbon Footprint (tCO2e/yr)' : 'Total Per-Household Carbon Footprint (tCO2e/yr)'}, inplace = True)
cfc_msa = cfc_msa.rename(columns = {"Population": "pop_2014", "Households": "hh_2014", 
                                    "electricity (kWh)": "electricity_kwh", "Nat. Gas (cu.ft.)": "nat_gas_cuft", 
                                    "FUELOIL (gallons)": "fueloil_gal", "Vehicle miles traveled": "vmt"})
cfc_msa.head()

,pop_2014,electricity_kwh,nat_gas_cuft,fueloil_gal,vmt,hh_2014,cf_transport_phh,cf_housing_phh,cf_food_phh,cf_goods_phh,cf_services_phh,cf_phh,cf
MSA,,,,,,,,,,,,,
"Abilene, TX",160828,35714.088774,98701.118579,1.336685,76234.634771,65492.0,15.927517,12.674110,7.667713,4.616458,5.004892,49.144098,3.218545e+06
"Akron, OH",678301,23257.037633,121455.109113,79.046948,47008.858596,160668.0,15.993913,11.746599,7.517472,5.391050,6.071775,50.968122,8.188946e+06
"Albany, GA",141811,38975.470766,85814.672587,9.162196,72787.504018,57134.0,14.360085,13.990202,7.972296,4.684970,5.050888,46.428323,2.652636e+06
"Albany-Lebanon, OR",108795,11139.121631,36096.695316,18.554493,26802.912183,44491.0,18.007041,10.646117,7.809633,4.995829,5.498125,54.317668,2.416647e+06
"Albany-Schenectady-Troy, NY",820825,36802.454287,233221.913105,1181.335835,103020.435963,392206.0,14.195390,10.778376,7.221763,5.349727,6.058857,47.021127,1.844197e+07


In [196]:
len(cfc_msa)

378

In [197]:
filepath = Path('../../data/tidy/msa-carbon-footprint.csv', index = False)
filepath.parent.mkdir(parents=True, exist_ok=True)  
cfc_df.to_csv(filepath)